In [1]:
import os
import torch
import random
import numpy as np
import networkx as nx
import pytorch_lightning as pl
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch_geometric.loader import DataLoader
from torch_geometric.utils import dense_to_sparse
from torch_geometric.nn import SimpleConv
torch.set_default_dtype(torch.float)
from utils.ClassicDistributedKalman import (centralized_extended_kalman_filter,
                                            diffusion_extended_kalman_filter_parallel_edge)
from utils.DistributedKalmanData import (HSystem, FSystem, GraphDataset, CreateGraph, 
                                         generate_data_points, generate_measurements)
from utils.DistributedKalmanNet import GraphKalmanProcess, EdgeKalmanFilter, GlobalKalmanFilter


In [2]:
def seed_everything(seed=42):
    """
    Set the random seed for reproducibility.
    :param seed: the random seed to set
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

In [15]:
def diffusion_extended_kalman_filter_parallel_edge2(measurements, f_system, h_system, r_array, q, p0, x0, j_matrix,
                                                   time_steps=10, node_num=10):
    simple_conv = SimpleConv(aggr='mean')
    edge_kalman_gain = EdgeKalmanFilter(h_system, node_num, r_array, 2)
    global_kalman_gain = GlobalKalmanFilter(h_system, node_num, r_array, 2)
    x_hat = np.zeros((time_steps, node_num, 2, 1))
    measurements = measurements.transpose(2, 0, 1)
    p = p0
    x_pred = x0
    for i in range(measurements.shape[0]):
        if i == 0:
            x_pred = x_pred[None, ...].repeat(node_num, axis=0)
        y_kalman_edge, _ = edge_kalman_gain(torch.tensor(x_pred, dtype=torch.float), torch.tensor(measurements[i, ...], dtype=torch.float), torch.tensor(j_matrix, dtype=torch.float))
        y_kalman_edge = y_kalman_edge[0, ...].numpy()
        m_all  = global_kalman_gain(torch.tensor(x_pred, dtype=torch.float), torch.tensor(p, dtype=torch.float), torch.tensor(j_matrix, dtype=torch.float))
        m_all = m_all[0, ...].numpy()
        x_current_all = x_pred + m_all @ y_kalman_edge
        edge_index, _ = dense_to_sparse(torch.tensor(j_matrix, dtype=torch.float))
        x_current = simple_conv(torch.tensor(x_current_all[..., 0], dtype=torch.float), edge_index)[..., None].numpy()
        p = kalman_process.gkf.calculate_p_mat(torch.tensor(x_current, dtype=torch.float), torch.tensor(m_all, dtype=torch.float), torch.tensor(j_matrix, dtype=torch.float))
        p = p[0, ...].numpy()
        x_pred = f_system.func(x_current)
        x_hat[i, ...] = x_current
    return x_hat

In [5]:
NODE_NUM = 100
TIME_STEPS = 100
torch.set_default_dtype(torch.float64)

In [6]:
h_sys = HSystem(node_num=NODE_NUM)
f_sys = FSystem()
g = nx.connected_watts_strogatz_graph(NODE_NUM, 5, 0.4, seed=42)
g = g.to_directed()
g.add_edges_from([(i, i) for i in range(NODE_NUM)])
adj_matrix = nx.adjacency_matrix(g).todense()

In [7]:
q = 1
P0 = 1 * np.eye(2)
x0 = np.array([[1, 1], ]).T
r_array = np.ones(NODE_NUM)

In [8]:
x_hat_kalman_list3 = []
x_hat_local_kalman_list = []
x_hat_diffusion_kalman_list = []
x_hat_alg_3_list = []
x_points_list = []

for i in tqdm(range(10)):
    x_points = generate_data_points(f_sys.func, q, np.array([10, 10],).T, TIME_STEPS)
    x_points_list.append(x_points.T)
    meas = generate_measurements(h_sys.func, x_points, r_array, n_expansions=1)
    print(meas.shape)
    print(x_points.shape)
    adj_matrix = nx.adjacency_matrix(g).todense()
    x_hat_kalman3 = centralized_extended_kalman_filter(meas, f_sys, h_sys, r_array, q, P0, x0, time_steps=TIME_STEPS, node_num=NODE_NUM)
    # x_hat_local_kalman = local_extended_kalman_filter(meas, f_sys, h_sys, r_array, q, P0, x0, adj_matrix, time_steps=TIME_STEPS, node_num=NODE_NUM)
    x_hat_diffusion_kalman = diffusion_extended_kalman_filter_parallel_edge(meas, f_sys, h_sys, r_array, q, P0, x0, adj_matrix, time_steps=TIME_STEPS, node_num=NODE_NUM)
    x_hat_alg_3 = diffusion_extended_kalman_filter_parallel_edge2(meas, f_sys, h_sys, r_array, q, P0, x0, adj_matrix, time_steps=TIME_STEPS, node_num=NODE_NUM)
    x_hat_kalman_list3.append(x_hat_kalman3)
    # x_hat_local_kalman_list.append(x_hat_local_kalman)
    x_hat_alg_3_list.append(x_hat_alg_3)
    x_hat_diffusion_kalman_list.append(x_hat_diffusion_kalman)
x_points = np.stack(x_points_list, axis=0)
x_hat_kalman3 = np.stack(x_hat_kalman_list3, axis=0)
x_hat_diffusion_kalman = np.stack(x_hat_diffusion_kalman_list, axis=0)
# x_hat_local_kalman = np.stack(x_hat_local_kalman_list, axis=0)
x_hat_alg_3 = np.stack(x_hat_alg_3_list, axis=0)

  0%|          | 0/10 [00:00<?, ?it/s]

(100, 1, 100)
(2, 100)


RuntimeError: expected scalar type Float but found Double

In [ ]:
err_kalman3 = np.linalg.norm(x_hat_kalman3 - x_points[..., None], axis=(-1, -2)).mean(axis=0)
# err_local_kalman = np.linalg.norm(x_hat_local_kalman - x_points[:, :, None, ..., None].repeat(NODE_NUM, axis=2),
#                                   axis=(-1, -2)).mean(axis=(0, 2))
err_local_kalman2 = np.linalg.norm(x_hat_diffusion_kalman - x_points[:, :, None, ..., None].repeat(NODE_NUM, axis=2),
                                   axis=(-1, -2)).mean(axis=(0, 2))
err_alg_3 = np.linalg.norm(x_hat_alg_3 - x_points[:, :, None, ..., None].repeat(NODE_NUM, axis=2), axis=(-1, -2)).mean(
    axis=(0, 2))

In [ ]:

x_points.shape

In [ ]:
x_hat_kalman3

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(err_kalman3, 'r', label='CEKF3')
# plt.plot(err_local_kalman, 'b', label='LDEKF')
plt.plot(err_alg_3, 'y', label='Alg-3')
plt.plot(err_local_kalman2, 'g', label='DiffEKF')
plt.legend()
plt.xlabel('Time Steps')
plt.ylabel('Estimation Error')
plt.ylim(0, 20)
plt.show()

 # Deep Extended Kalman Filter

In [3]:
seed_everything(42)
NODE_NUM = 50
TIME_STEPS=10
q = 1
P0 = 1 * np.eye(2)
x0 = np.array([[1, 1], ]).T
r_array = np.ones(NODE_NUM)
h_sys = HSystem(node_num=NODE_NUM)
f_sys = FSystem()
g = CreateGraph(NODE_NUM, 5)
train_dataset = GraphDataset(g, f_sys, h_sys, q, r_array, monte_carlo_simulations=10000, time_steps=TIME_STEPS, n_expansions=1)  
val_dataset = GraphDataset(g , f_sys, h_sys, q, r_array, monte_carlo_simulations=100, time_steps=TIME_STEPS, n_expansions=1, mean=train_dataset.mean, std=train_dataset.std)
# train_dataset_overfit
# train_dataset_overfit2 = [train_dataset[1] for _ in range(10000)]

In [4]:
seed_everything(42)
BATCH_SIZE = 32
train_dataset_overfit_batch = [[train_dataset[i] for i in range(BATCH_SIZE)] for _ in range(100)]
train_dataset_overfit = [example for nested in train_dataset_overfit_batch for example in nested]
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
train_loader_overfit = DataLoader(train_dataset_overfit, shuffle=False, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=BATCH_SIZE)

In [5]:
seed_everything(42)
kalman_process = GraphKalmanProcess(
    NODE_NUM, h_sys, f_sys, signal_dim=2, node_feature_dim=4, node_output_dim=16,
    edge_features_dim=1, node_kalman_dim=4, edge_kalman_dim=2, hidden_dim=32, heads=16, 
    lr=0.0005, r_array=r_array)
kalman_process = kalman_process.to(torch.float)

In [6]:
# take first batch of train_loader
for idx, batch in enumerate(train_loader):
    print(batch[0].x.shape[0])    
    break

50


In [7]:
# create pl trainer
for name, param in kalman_process.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

gkf.gat_rnn.fc_delta_y_innov.0.weight torch.Size([32, 3])
gkf.gat_rnn.fc_delta_y_innov.0.bias torch.Size([32])
gkf.gat_rnn.fc_delta_y_innov.2.weight torch.Size([32, 32])
gkf.gat_rnn.fc_delta_y_innov.2.bias torch.Size([32])
gkf.gat_rnn.fc_r.0.weight torch.Size([32, 1])
gkf.gat_rnn.fc_r.0.bias torch.Size([32])
gkf.gat_rnn.fc_r.2.weight torch.Size([32, 32])
gkf.gat_rnn.fc_r.2.bias torch.Size([32])
gkf.gat_rnn.r_input_gru.weight_ih_l0 torch.Size([96, 32])
gkf.gat_rnn.r_input_gru.weight_hh_l0 torch.Size([96, 32])
gkf.gat_rnn.r_input_gru.bias_ih_l0 torch.Size([96])
gkf.gat_rnn.r_input_gru.bias_hh_l0 torch.Size([96])
gkf.gat_rnn.gcn_nodes.bias torch.Size([32])
gkf.gat_rnn.gcn_nodes.lin.weight torch.Size([32, 64])
gkf.gat_rnn.fc_signal_features.0.weight torch.Size([32, 4])
gkf.gat_rnn.fc_signal_features.0.bias torch.Size([32])
gkf.gat_rnn.fc_signal_features.2.weight torch.Size([32, 32])
gkf.gat_rnn.fc_signal_features.2.bias torch.Size([32])
gkf.gat_rnn.sigma_gru.weight_ih_l0 torch.Size([12, 32

In [8]:
seed_everything(42)
# define early stopping for validation loss
early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss:_epoch', patience=5, verbose=True, mode='min', )
trainer = pl.Trainer(max_epochs=2000, accelerator='auto', log_every_n_steps=1, callbacks=[early_stopping])
# trainer = pl.Trainer(max_epochs=2000, accelerator='auto', log_every_n_steps=1)
trainer.fit(kalman_process, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\Lenovo\anaconda3\envs\thesis\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name | Type              | Params | Mode 
---------------------------------------------------
0 | gkf  | GraphKalmanFilter | 17.8 K | train
1 | loss | MSELoss           | 0      | train
---------------------------------------------------
17.8 K    Trainable params
0         Non-trainable params
17.8 K    Total params
0.071     Total estimated

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\Lenovo\anaconda3\envs\thesis\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
C:\Users\Lenovo\anaconda3\envs\thesis\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss:_epoch did not improve in the last 5 records. Best score: 0.521. Signaling Trainer to stop.


In [10]:
len(val_dataset)

100

In [11]:
val_dataset[0]

Data(x=[50, 10, 1], edge_index=[2, 150], edge_attr=[150, 1], y=[10, 2, 1], adj_matrix=[50, 50])

In [23]:
x_points_list = []
x_hat_alg_3_list = []
for graph in tqdm(val_dataset):
    x_points = graph.y[..., 0].numpy().T
    x_points_list.append(x_points)
    meas = graph.x.numpy().transpose(0, 2, 1)
    adj_matrix = graph.adj_matrix.numpy()
    x_hat_alg_3 = diffusion_extended_kalman_filter_parallel_edge(meas, f_sys, h_sys, r_array, q, P0, x0, adj_matrix, time_steps=10, node_num=50)
    # x_hat_local_kalman_list.append(x_hat_local_kalman)
    x_hat_alg_3_list.append(x_hat_alg_3)
x_points = np.stack(x_points_list, axis=0)

x_hat_alg_3 = np.stack(x_hat_alg_3_list, axis=0)

  0%|          | 0/100 [00:00<?, ?it/s]

In [24]:
x_hat_alg_3

array([[[[[ 4.42215614],
          [ 5.76468666]],

         [[ 4.32198179],
          [ 5.9706175 ]],

         [[ 4.40909669],
          [ 5.57425806]],

         ...,

         [[ 3.96074681],
          [ 6.22785156]],

         [[ 4.60911746],
          [ 5.4331455 ]],

         [[ 4.64634212],
          [ 5.74275514]]],


        [[[ 7.20278365],
          [ 8.56923005]],

         [[ 7.40803924],
          [ 8.88303809]],

         [[ 7.36503762],
          [ 8.6354478 ]],

         ...,

         [[ 7.23696148],
          [ 8.58159905]],

         [[ 7.83749417],
          [ 9.21280109]],

         [[ 7.37320906],
          [ 8.75473238]]],


        [[[ 9.81168678],
          [ 9.41778309]],

         [[ 9.91938351],
          [ 9.72692957]],

         [[ 9.62989279],
          [ 9.46891511]],

         ...,

         [[ 9.90278176],
          [ 9.50729321]],

         [[ 9.89606053],
          [ 9.58404344]],

         [[ 9.89517395],
          [ 9.68435307]]],


        ...,


In [30]:
err_alg_3 = np.linalg.norm(x_hat_alg_3 - x_points.transpose(0, 2, 1)[:, :, None, ..., None].repeat(NODE_NUM, axis=2), axis=(-1, -2)).mean(axis=(0, 2))

In [36]:
err_alg_3[1:]

array([2.12737906, 0.64501009, 0.4994892 , 0.45502138, 0.4706144 ,
       0.48465942, 0.4644399 , 0.43621632, 0.47314097])

In [28]:
x_points.transpose(0, 2, 1).shape

(100, 10, 2)

In [29]:
x_points[0, ...]

array([[ 9.93004533,  9.30773961, 10.07219948, 10.99209763,  9.7579503 ,
         9.19677078, 11.00202033, 10.76947583,  9.325453  ,  9.96717477],
       [ 9.4509761 ,  8.9590512 ,  9.65008575,  7.51339912,  6.73104154,
         6.60178819,  5.90219713,  5.84460649,  4.5119292 ,  2.11965037]])

In [49]:
trainer.predict(kalman_process, val_loader)

Predicting: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [41]:
trainer.predict(kalman_process, val_loader)

Predicting: |          | 0/? [00:00<?, ?it/s]

RuntimeError: shape '[5, 50, 10, 1]' is invalid for input of size 500

In [45]:
1+1

2

In [ ]:
0